# Collaborative Filtering

**THIS IS ONLY FOR 15-688 STUDENTS**

In this question you will use collaborative filtering to make movie recommendations. The purpose of this question is to become familiar with recommendation systems, how they train and how they create recommendations.

## Collaborative Filtering by Matrix Factorization

In collaborative filtering, we take the matrix of user ratings and attempt to factorize it into user-related features and movie-related features. By viewing ratings as the product of both user and movie properties, we can predict user ratings on movies they have not yet watched.

In more formal terms, given some partially filled ratings matrix $X\in \mathbb{R}^{m\times n}$, we want to find feature matrices $U \in \mathbb{R}^{m\times k}$ and $V \in \mathbb{R}^{n\times k}$ such that $UV^T = X$. Each row of $U$ is a feature-vector corresponding to a user, and each row of $V$ is a feature-vector corresponding to a movie. $u_i^Tv_j$ is the predicted rating of user $i$ on movie $j$. This is our hypothesis function for collaborative filtering:

$$h_\theta(i,j) = u_i^T v_j$$


$X$ is usually quite sparse, so we can indicate the absence of a rating with the value 0. Let $S$ be the set of $(i,j)$ such that $X_{i,j} \neq 0$, so $S$ is the set of all pairs for which we have a rating. When factorizing $X$ into $U$ and $V$, we wish to minimize the squared loss between our hypothesis for $(i,j) \in S$ and the actual value:

$$\ell\left(h_\theta(i,j),X_{i,j}\right) = \left(h_\theta(i,j) - X_{i,j}\right)^2\qquad\forall (i,j) \in S$$

The total loss is the sum of these individual losses and an additional $l_2$ penalty on the parameters (for regularization), so our total loss is:

$$\sum_{i,j\in S}\ell(h_\theta(i,j),X_{i,j}) + \lambda_u ||U| |_2^2 + \lambda_v ||V||_2^2$$

For this assignment, we'll let the regularizing term weights be $\lambda_u = \lambda_v = \lambda$. 

In [2]:
import os
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
import matplotlib
import gzip
import csv
import time
from collections import Counter
from sklearn.model_selection import train_test_split

matplotlib.use("svg")
if not os.environ.get("DISABLE_TESTING", False):
    %matplotlib inline

import matplotlib.pyplot as plt
plt.style.use("ggplot")

from testing.testing import test

## MovieLens rating dataset

We will be using the MovieLens small dataset. The original dataset has over 27 million ratings; we will use only 100k ratings. You can read about the dataset [here](https://grouplens.org/datasets/movielens/).

For this assignment, we will only be looking at the ratings data, and ignoring not their movie data and user made tags for movies) which could be used to improve the ability of the recommendation system. We begin by giving you some code to read the dataset:

In [3]:
def read_csv(fn_ratings="ratings.csv.gz", fn_movies="movies.csv.gz"):
    """read the GZipped CSV data and split it into headers and newlines.

    kwargs:
        fn_ratings : str -- file with ratings
        fn_movies : str -- file with movie names and ids
    
    returns: Tuple[ratings, movies] where
      ratings : Tuple[np.ndarray[int], np.ndarray[int], np.ndarray[float]] -- a list of user ids, movie ids, and corresponding ratings
      movies : Dict[int, str] -- the lookup table from movie ID to movie name
    """
    
    movies = {}
    ratings_userid  = []
    ratings_movieid = []
    ratings_rating  = []
    with gzip.open(fn_movies, 'rt', newline="", encoding='utf-8') as f:
        csvobj = csv.reader(f)
        assert tuple(next(csvobj)) == ("movieId", "title")
        for row in csvobj:
            movies[int(row[0])] = row[1]

    with gzip.open(fn_ratings, 'rt', newline="", encoding='utf-8') as f:
        csvobj = csv.reader(f)
        assert tuple(next(csvobj)) == ("userId", "movieId", "rating")
        for row in csvobj:
            ratings_userid.append(int(row[0]))
            ratings_movieid.append(int(row[1]))
            ratings_rating.append(float(row[2]))

    return (np.array(ratings_userid) - 1, np.array(ratings_movieid), np.array(ratings_rating)), movies

## Data preparation

Matrix factorization requires that we have our ratings stored in a matrix of users, so the first task is to take the dataframe and convert it into this format. Typically these matrices are extremely large and sparse (especially if you want to process the 24 million ratings), and so we work with sparse matrices here. 

### Specification

You should produce a ratings matrix and a movie lookup dictionary, where:
* Each row of the ratings matrix corresponds to a user. (The user IDs already form a consecutive range of numbers.)
* Each column of the ratings matrix corresponds to a movie. The order of the columns doesn't matter. The movie IDs do not form a consecutive range of numbers, so you need to change the movie IDs to make them consecutive.
* Any entry that does not have a rating should have a default value of 0. 
* You should produce a new movie lookup dictionary that correctly maps your updated movie IDs to the movie title. Any movies not present in the dataset should be omitted from the new lookup dictionary.
* Split the data using [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html); you should be splitting by individual _ratings_ and not by users. (That is, different ratings from the same user are allowed to appear in the training set and test set.)

In [4]:
from scipy.sparse import coo_matrix

def process_test(process):
    ratings, movies_original = read_csv()
    X_tr, X_te, movies = process(*ratings, movies_original)
    
    # Check format of created arrays:
    test.equal(repr(X_tr), """<610x9724 sparse matrix of type '<class 'numpy.float64'>'
	with 90752 stored elements in COOrdinate format>""")
    test.equal(repr(X_te), """<610x9724 sparse matrix of type '<class 'numpy.float64'>'
	with 10084 stored elements in COOrdinate format>""")

    # Correct number of movies:
    test.equal(len(movies), X_tr.shape[1])
    # No new movie titles:
    test.equal(len(set(movies) - set(movies_original.values())), 0)
    # 18 movies that don't appear in the dataset and should be removed:
    test.equal(len(set(movies_original.values()) - set(movies)), 18)

@test
def process(ratings_userid, ratings_movieid, ratings_rating, movies, test_size=0.1, random_state=0xCAFE):
    """ Given rating data, split the data into training and testing sets and convert them to sparse matrices.

        args: 
            ratings_userid  : np.ndarray[num_ratings] -- vector of user Ids
            ratings_movieid : np.ndarray[num_ratings] -- vector of movie Ids
            ratings_rating  : np.ndarray[num_ratings] -- vector of rating values
            movies  : Dict[int, str] -- lookup from movie Id to movie name

        kwargs:
            test_size : float -- fraction of dataset to place in the test set
            random_state : int -- the random seed for dataset splitting

        return: Tuple[X_train, X_test, movies] 
            X_train : sp.coo_matrix -- the training data, as a sparse matrix
            X_test : sp.coo_matrix -- the test data, as a sparse matrix
            movies  : List[str] -- list of movie names, each at position equal to the new movie Id. 
    """
    
    old_new = {}
    new_movies = {}
    count = 0
    for old_id in ratings_movieid:
        if (old_id not in old_new.keys()):
            old_new[old_id] = count
            new_movies[count] = movies[old_id]
            count += 1
    new_id = [old_new[old_id] for old_id in ratings_movieid]
    ratings_newmovieid = np.asarray(new_id)
    new_movies_list = [new_movies[i] for i in new_movies.keys()]
    U_train, U_test, V_train, V_test, R_train, R_test = train_test_split(ratings_userid, ratings_newmovieid, ratings_rating, test_size=test_size, random_state=42)
    
    m = len(set(ratings_userid))
    n = len(new_movies)
    X_train = coo_matrix((R_train, (U_train, V_train)), shape = (m,n))
    X_test = coo_matrix((R_test, (U_test, V_test)), shape = (m,n))
    
    return X_train, X_test, new_movies_list


### TESTING process: PASSED 5/5
###



## Alternating Minimization for Collaborative Filtering
Now we build the collaborative filtering recommendation system. We will use a method known as alternating least squares. We alternate between optimizing $U$ and $V$ and holding the other constant. By treating one matrix as a constant, we get a weighted least squares problem which we can solve easily. More details can be found in the lecture notes.

We begin by writing a function to calculate the error:

In [5]:
def error_test(error):
    U = np.array([[.1, .2, .3, .4, .5]])
    V = np.array([[1., 2., 3., 4., 5.], [3., 4., 5., 6., 7.]])
    X = np.array([[5., 0]])
    
    test.equal(error(X, U, V), 0.25)

@test
def error(X, U, V):
    """ Compute the mean error of the observed ratings in X and their estimated values. 

        args: 
            X : np.array[num_users, num_movies] -- the ratings matrix
            U : np.array[num_users, num_features] -- a matrix of features for each user
            V : np.array[num_movies,num_features] -- a matrix of features for each movie

        return: float -- the mean squared error between non-zero entries of X and the ratings
            predicted by U and V; as this is an error and not a loss function, you do not need to include the
            regularizing terms.
        """
    
    A = U @ V.T
    m = U.shape[0]
    n = V.shape[0]
    count = 0
    error = 0
    for i in range(m):
        for j in range(n):
            if(X[i,j]!=0):
                error += (A[i,j]-X[i,j]) * (A[i,j]-X[i,j])
                count += 1
                
    return error/count

### TESTING error: PASSED 1/1
###



Now we write the training function.

### Specification

* There is a verbose parameter here; if true, you should evaluate and print the training and test error every few steps. These should decrease (and converge).
* You may find it useful to have an indicator matrix W where $W_{ij} = 1$ if there is a rating in $X_{ij}$. 
* You can initialize U, V with values distributed [normally](numpy.random.normal).
* Assume inputs are dense.
* The time limits assume a modern laptop; you can ignore them as long as they pass on the server. (Error 143 

Hints:

* You should be iterating over each row of V and updating it while holding U constant, and then iterating over each row of U while holding V constant.
* You should use la.solve (scipy.linalg.solve) to update each row.
* The most challenging part of this assignment is writing 

In [49]:
def train_test(train):
    ratings, movies_original = read_csv()
    X_tr, X_te, movies = process(*ratings, movies_original)
    X_tr = X_tr.toarray() # The matrix is small enough we can use the dense form.
    X_te = X_te.toarray()
    
    # Test with k=1
    start_time = time.perf_counter()
    U, V = train(X_tr, X_te, 1, niters=12)
    test.true(time.perf_counter() - start_time < 60)
    test.true(error(X_tr, U, V) < 0.9)
    test.true(error(X_te, U, V) < 1.4)

    # Test with k=3
    start_time = time.perf_counter()
    U, V = train(X_tr, X_te, 3, niters=12)
    test.true(time.perf_counter() - start_time < 120)
    test.true(error(X_tr, U, V) < 0.8)
    test.true(error(X_te, U, V) < 1.4)
    
    
@test
def train(X_train, X_test, k, niters=12, lam=10., verbose=True):
    """ Train a collaborative filtering model. 
        Args: 
            X_train : np.array[num_users, num_movies] -- the training ratings matrix, assumed dense
            X_test : np.array[num_users, num_movies] -- the test ratings matrix, assumed dense
            k : int -- the number of features in the CF model
            niters : int -- number of iterations to run
            lam : float -- regularization parameter, shown as lambda
            verbose : boolean -- if true, print the error on train and test sets every few iterations 

        return : Tuple[U, V]
            U : np.array[num_users,  num_features] -- the user-feature matrix
            V : np.array[num_movies, num_features] -- the movie-feature matrix
    """

#     print(X_train.shape)
#     print(k)
    m,n = X_train.shape
    W_train = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            if X_train[i,j] != 0:
                W_train[i,j] = 1

    U = np.random.normal(size = (m, k))*5
    V = np.random.normal(size = (n, k))*5

    if verbose:
        print("| Time    | Iter  | Train Err | Test Err |")
        print("| ------- | ----- | --------- | -------- |")

    start_time = time.perf_counter()
    for i in range(niters):
        for u in range(m):
            V_ = V * W_train[u,:][:,None]
            a = (np.dot(V_.T, V_))
            b = (np.dot(X_train[u,:], V))
            lamda = np.eye(k) * lam
            U[u,:] = la.solve(a + lamda, b)
        
        for v in range(n):
            U_ = U * W_train[:,v][:,None]
            a = (np.dot(U_.T, U_))
            b = (np.dot(X_train[:,v], U))
            lamda = np.eye(k) * lam
            V[v,:] = la.solve(a + lamda, b)
        
        if verbose: 
            print(f"| {time.perf_counter() - start_time: 7.3f} |{i+1: 6d} |{error(X_train, U, V):10.4f} |{error(X_test, U, V):9.4f} |")
    
    if verbose: 
        print("")
    return U, V

| Time    | Iter  | Train Err | Test Err |
| ------- | ----- | --------- | -------- |
|   0.917 |     1 |   13.2762 |  13.2379 |
|   6.445 |     2 |    1.7887 |   2.2300 |
|  11.807 |     3 |    0.9512 |   1.4484 |
|  17.050 |     4 |    0.9216 |   1.4241 |
|  22.811 |     5 |    0.9112 |   1.4170 |
|  28.051 |     6 |    0.9043 |   1.4126 |
|  33.398 |     7 |    0.8990 |   1.4092 |
|  40.090 |     8 |    0.8947 |   1.4066 |
|  45.966 |     9 |    0.8913 |   1.4046 |
|  51.819 |    10 |    0.8885 |   1.4030 |
|  56.943 |    11 |    0.8862 |   1.4017 |
|  62.241 |    12 |    0.8844 |   1.4006 |

| Time    | Iter  | Train Err | Test Err |
| ------- | ----- | --------- | -------- |
|   1.299 |     1 |   12.9417 |  12.9507 |
|   7.070 |     2 |    1.4594 |   2.1017 |
|  12.996 |     3 |    0.9518 |   1.6283 |
|  18.863 |     4 |    0.8485 |   1.5344 |
|  24.895 |     5 |    0.8043 |   1.4899 |
|  30.710 |     6 |    0.7797 |   1.4630 |
|  36.629 |     7 |    0.7639 |   1.4455 |
|  42.460 

We only need to train for 12 iterations, which should be quick.

## Recommendations

Finally, we need to be able to make recommendations given a matrix factorization. We can do this by simply taking the recommending the movie with the highest value in the estimated ratings matrix. 

### Specification
* For each user, recommend the the movie with the highest predicted rating for that user that the user **hasn't** seen before. 
* Return the result as a list of movie Ids. 

In [55]:
def recommend_test(recommend):
    U = np.array([[1., 1., 0., 0.], [0., 1., 0., 1.]])
    V = np.array([[.5, 0., 0., .5], [0., .5, .5, 0.], [.5, -1., .5, 3.]])
    X = np.array([[5., 0., 0.], [1., 1., 0.]])
    recommendation = recommend(X, U, V)
    test.true(isinstance(recommendation, list))
    test.equal(recommendation, [1, 2])
    
    # Print the five most commonly recommended movies.
    # This part is not graded.
    ratings, movies_original = read_csv()
    X_tr, X_te, movies = process(*ratings, movies_original)
    X_tr = X_tr.toarray()
    X_te = X_te.toarray()
    U, V = train(X_tr, X_te, 3, niters=4)
    recommendation = recommend(X_tr + X_te, U, V)
    counts = Counter(recommendation)
    print([(movies[i], c) for i, c in counts.most_common(5)])

@test
def recommend(X, U, V):
    """Recommend a new movie for every user.

        args: 
            X : np.array[num_users, num_movies] -- the ratings matrix
            U : np.array[num_users, num_features] -- a matrix of features for each user
            V : np.array[num_movies,num_features] -- a matrix of features for each movie

        return: List[int] -- a list of movie Ids for each user
    """
    m,n = X.shape
    W = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            if X[i,j] == 0:
                W[i,j] = 1
                
    score = W * (U @ V.T)

    return [i.tolist().index(max(i)) for i in score]

| Time    | Iter  | Train Err | Test Err |
| ------- | ----- | --------- | -------- |
|   1.340 |     1 |   13.0730 |  13.0800 |
|   7.067 |     2 |    1.2560 |   1.8544 |
|  12.605 |     3 |    0.8634 |   1.4907 |
|  18.208 |     4 |    0.7997 |   1.4498 |

[('Shawshank Redemption, The (1994)', 253), ("Schindler's List (1993)", 47), ('Forrest Gump (1994)', 31), ('Usual Suspects, The (1995)', 28), ('Pulp Fiction (1994)', 26)]
### TESTING recommend: PASSED 2/2
###



Our implementation tends to recommend popular movies, which yours should as well. The exact results vary based on the random spliting of training and test sets.

```
('Shawshank Redemption, The (1994)', 236),
("Schindler's List (1993)", 87),
('Streetcar Named Desire, A (1951)', 47),
('Usual Suspects, The (1995)', 41),
('Forrest Gump (1994)', 37)
```